In [1]:
import os, requests, time

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from urllib.parse import urlparse, urljoin
import csv
import pandas as pd

## Make lists of URLs per country

### NL

In [3]:
nl_files_list = os.listdir("../met-naam-2018/castles-netherlands/")

In [79]:
dchar_domain_extenstions = ['net','org','com','top']
nl_urls_div = [link[:-4] for link in nl_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
nl_urls_div = ['https://www.'+link[:-3] + '.' + link[-3:] for link in nl_urls_div]

nl_urls_nl = ['https://www.'+item[:-6]+'.'+item.rstrip('.txt')[-2:] for item in nl_files_list]

nl_urls = nl_urls_div + nl_urls_nl

In [80]:
print(len(nl_urls), nl_urls[-1:])

58 ['https://www.artlandt.op']


In [82]:
nl_urls[10:20]

['https://www.slotzuylen.nl',
 'https://www.kasteelgeldrop.nl',
 'https://www.fraeylemaborg.nl',
 'https://www.slotloevestein.nl',
 'https://www.museumijsselstein.nl',
 'https://www.twickel.nl',
 'https://www.kasteelduivenvoorde.nl',
 'https://www.huystendonck.nl',
 'https://www.kasteeldussen.nl',
 'https://www.huisbergh.nl']

In [19]:
print(len(nl_urls), nl_urls[-1:])

58 ['https://www.artlandt.op']


In [21]:
# optional: save this list as a csv file
df_nl = pd.DataFrame(data={"NL domain": nl_urls})
df_nl.to_csv("../url_lists/nl_urls.csv", sep=',',index=False)

### FR

In [179]:
fr_files_list = os.listdir("../met-naam-2018/castles-france/")
len(fr_files_list)

92

In [180]:
domain_extensions = ['com', 'net', 'org', 'bzh']
fr_urls_div = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-7:-4] in domain_extensions]
fr_urls_div = ['http://www.'+link[:-3] + '.' + link[-3:] for link in fr_urls_div]


fr_urls_fr = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-6:-4]== 'fr']
fr_urls_fr = ['http://www.'+link[:-2] + '.' + link[-2:] for link in fr_urls_fr]

#catch the unusual .corsica domains
fr_urls_corsica = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-11:-4]== 'corsica']
fr_urls_corsica = ['http://'+link[:-7] + '.' + link[-7:] for link in fr_urls_corsica]

fr_urls = fr_urls_div + fr_urls_fr + fr_urls_corsica
print(len(fr_urls))

91


In [181]:
# optional: save this list as a csv file
df_fr = pd.DataFrame(data={"FR domain": fr_urls})
df_fr.to_csv("../url_lists/fr_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_fr)

### DE

In [185]:
de_files_list = os.listdir("../met-naam-2018/castles-germany/")
len(de_files_list)

90

In [186]:
dchar_domain_extenstions = ['net','org','com']
de_urls_div = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
de_urls_div = ['http://www.'+link[:-3] + '.' + link[-3:] for link in de_urls_div]

cchar_domain_extensions = ['de','nl','eu']
de_urls_de = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-6:-4] in cchar_domain_extensions]
de_urls_de = ['http://www.'+link[:-2] + '.' + link[-2:] for link in de_urls_de]

#catch the .info domain
de_urls_info = [link[:-4] for link in de_files_list if link.endswith('.txt') and link[-8:-4]== 'info']
de_urls_info = ['http://'+link[:-4] + '.' + link[-4:] for link in de_urls_info]

de_urls = de_urls_div + de_urls_de + de_urls_info
print(len(de_urls))

90


In [191]:
# optional: save this list as a csv file
df_de = pd.DataFrame(data={"DE domain": de_urls})
df_de.to_csv("../de_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_de)

### UK

In [229]:
uk_files_list = os.listdir("../met-naam-2018/castles-united-kingdom/")
len(uk_files_list)

76

In [230]:
dchar_domain_extenstions = ['net','org','com']
uk_urls_div = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-7:-4] in dchar_domain_extenstions]
uk_urls_div = ['https://www.'+link[:-3] + '.' + link[-3:] for link in uk_urls_div]

cchar_domain_extensions_5 = ['govuk','orguk']
uk_urls_gov = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-9:-4] in cchar_domain_extensions_5]
uk_urls_gov = ['https://www.'+link[:-5] + '.' + link[-5:-2] + '.' + link[-2:] for link in uk_urls_gov]

cchar_domain_extensions_4 = ['couk','acuk']
uk_urls_co = [link[:-4] for link in uk_files_list if link.endswith('.txt') and link[-8:-4] in cchar_domain_extensions_4]
uk_urls_co = ['https://www.'+link[:-4] + '.' + link[-4:-2] + '.' + link[-2:] for link in uk_urls_co]

uk_urls = uk_urls_div + uk_urls_gov + uk_urls_co + ['https://www.hemyockcastle.uk']
print(len(uk_urls))

76


In [232]:
# optional: save this list as a csv file
df_uk = pd.DataFrame(data={"UK domain": uk_urls})
df_uk.to_csv("../uk_urls.csv", sep=',',index=False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print (df_uk)

## URL Check up functions 

In [5]:
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(0)
        else:
            print(f"The URL '{url}' is unreachable. Status code: {response.status_code}")
    except requests.ConnectionError:
        print(f"The URL '{url}' is unreachable.")

In [6]:
cc = 'FR'

In [8]:
# optional: change between existing url list (made in this notebook) and a locally stored csv list 
domain = pd.read_csv('../url_lists/'+cc+'_urls.csv')
urls = domain['FR domains'].values.tolist() 

for i in range(len(urls)):
    #print(urls[i])
    check_url(urls[i])

The URL 'https://www.la-hunaudaye.com/' is unreachable. Status code: 403
0
0
The URL 'https://www.chateau-eaucourt.com' is unreachable. Status code: 429
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateau-pirou.fr/' is unreachable. Status code: 410
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateau-picquigny.com' is unreachable. Status code: 429
The URL 'https://www.chateau-bazoches.com' is unreachable.
0
0
0
0
0
0
0
The URL 'https://www.chateau-malbrouck.com' is unreachable.
0
0
0
The URL 'https://www.chateau-de-la-rochepot.com' is unreachable.
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateaugavray.fr' is unreachable. Status code: 555
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateaudetanlay.fr' is unreachable. Status code: 429
The URL 'https://www.folleville-chateau-medieval.fr' is unreachable. Status code: 429
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## Notes on URLS: 

#### DUTCH CASTLES (XX/55)
- https://www.museumdefundatie.nl/ is not on the 2018 list
- https://www.beeldentuinravesteyn.nl changed domain to: https://www.ruine-ravesteyn.nl/

#### FRENCH CASTLES (6/91)
doesn't respond to checkup function (errors 429, 403 and 555) but the link works - it is correct: 
- https://www.chateau-eaucourt.com/ 
- http://www.chateau-picquigny.com
- https://www.la-hunaudaye.com/
- https://www.chateaudetanlay.fr/
- https://www.chateaugavray.fr/

seems to be closed, link won't open but WHOIS listing exists:
- www.chateau-de-la-rochepot.com (wikipedia: https://en.wikipedia.org/wiki/Ch%C3%A2teau_de_la_Rochepot "In October 2018, the castle was seized by the French government after investigation into an alleged money laundering scheme by Dmytro Malynovskyi, a Ukrainian. Malynovski was arrested after an investigation into a corruption and money laundering scheme in which he purchased and lived in the castle after faking his own death.")
 
#### GERMAN CASTLES (10/90)
doesn't respond to checkup function (errors 429 and 403) but the link is correct:
- https://www.schlossparkmarisfeld.de/
- https://www.schloss-marienburg.de/
- https://schloss-langenburg.de/
- https://schloss-karlsberg.de/ (secure connection missing or required?)
- http://www.schloss-braunfels.de'

possible mistakes and changes to domain names:
- wwwkasteelbentheimnl -> https://burg-bentheim.de/ text is in dutch in German 2018 scraped material
- http://evenburg.landkreis-leer.de/ redirects to http://landkreis-leer.de/ the castle does not have its own domain since 2022, review the subdomain  wmk.landkreis-leer.de/ as replacement castle representation
- http://www.wilhelmshoehede.txt requires Flash (very outdated site, no accessible content)
- https://www.schloesser-hessen.de/de/schloss-erbach is a collection of castles? thus https://www.schloss-erbach.de redirects here
- http://burghohnstein.com is for sale, it is replaces by http://burghohnstein.info which is included in the list here

#### UK CASTLES (12/76)
doesn't respont to checkup function (error 429, 403):
- https://www.hrp.org.uk/ - is a collection of palaces - perhaps there are individual websites?
- https://www.hants.gov.uk/
- https://www.haddonhall.co.uk/
- https://www.tamworthcastle.co.uk/
- https://www.whittingtoncastle.co.uk only reachable without 'www' (as https://whittingtoncastle.co.uk/)

possible mistake:
- why is https://dur.ac.uk/ on the list? it is the university website, no (direct) castle content

closed:
- https://wresslecastle.org/ domain not maintained (people speak of using Facebook page to inquire about opening hours), Selena removed from url list Nov 2024
- https://www.leicestercastle.co.uk domain not maintained, information on the castle available on history of Leicestre website, https://www.storyofleicester.info/

changed domains:
- http://www.rabycastle.com is now https://www.raby.co.uk 
- http://www.castlerising.com is now http://www.castlerising.co.uk
- http://www.fobt.org.uk is now https://friendsofbuckdentowers.co.uk/
- https://www.turtontower.co.uk is now https://turtontower.com/




## Notes for writing
- visit https://castlestudiestrust.org/ and understand what they mean by 'cutting edge castle research'
- narrate how domains differ for countries - NL are all .nl except for 2 .com, while FR, DE and UK have different endings; this incoherency is reflected in the different character of top-level domain endings - some FR, DE and UK casltes have the commercial .com and co.uk ending, while others go with .org and .net which are in principle for non-profit organisations; .de and .fr (and 1 .uk) domains reflect the standard country association, while only some UK castles have gov.uk castles, directly related to the government
- General questions:
1. what is a web presentation of the castle, what cultural meaning does it cary, how is digital communication important to castle preservation and management?
 